# Transferring a model from PyTorch into Caffe2 using ONNX

In this tutorial, we describe how to use ONNX to convert a model defined in PyTorch into the ONNX format and then load it into Caffe2. Once in Caffe2 we can run the model to double check it was exported correctly, and then use Caffe2 features such as exporting the model for execution on mobile devices.

This work is a joint collaboration among various awesome developers on PyTorch and Caffe2.

In [2]:
# Import a bunch of stuff we are going to need for this
import io
import numpy as np

from torch import nn
from torch.autograd import Variable
import torch.utils.model_zoo as model_zoo
import torch.onnx

import onnx
from onnx.backend import c2

For this example, we will transfer a super resolution model that can increase the resolution of photographs. First, let's create a SuperResolution model in PyTorch. [This model](https://github.com/pytorch/examples/blob/master/super_resolution/model.py) comes directly from PyTorch's examples without modification:

In [3]:
# Super Resolution model definition in PyTorch
import torch.nn as nn
import torch.nn.init as init


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal(self.conv4.weight)

# Create the super-resolution model by using the above model definition.
py_model = SuperResolutionNet(upscale_factor=3)

We next download some pre-trained weights for this model to avoid having to train a model for this tutorial.

In [4]:
# define input, load pretrained model weights
model_url = 'https://s3.amazonaws.com/pytorch/test_data/export/superres_epoch100-44c6958e.pth'
batch_size = 2    # just a random number

# Initialize model with the pretrained weights
py_model.load_state_dict(model_zoo.load_url(model_url))

# set the train mode to false since we will only run the forward pass.
py_model.train(False)

SuperResolutionNet (
  (relu): ReLU ()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pixel_shuffle): PixelShuffle (upscale_factor=3)
)

Exporting a model in PyTorch works via tracing. To export a model, you call the `torch.onnx.export()` function which executes the model, recording a trace of what operators are used to compute the outputs. Because `export` runs the model, we need provide an input tensor `x`. The values in this tensor are not important, but it needs to be the right size.

In [5]:
# Input to the model
x = Variable(torch.randn(batch_size, 1, 224, 224), requires_grad=True)

# Export the model 
torch_out = torch.onnx.export(py_model, # model being run 
                              x, # the input to the model (or a tuple for multiple inputs) 
                              "super_resolution.onnx", # where to save the model (can also be a file or file-like object)
                              export_params=True) # store the trained parameter weights inside the model file

`torch_out` is the output after executing the model. Normally you can ignore this output, but here we will use it to verify that the model we exported computes the same values when run in Caffe2.


Now let's take this ONNX model and use it in Caffe2. This part can normally be done in seperate process or on another machine, but we will continue in the same process so that we can verify that Caffe2 and PyTorch are computing the same value for the network:

In [6]:
# Load the ONNX GraphProto object.
# graph is a standard Python protobuf object
graph = onnx.load("super_resolution.onnx")

# prepare the caffe2 backend for executing the model
# this converts the ONNX graph into a Caffe2 NetDef 
# that can execute it
prepared = c2.prepare(graph)

# run the model in Caffe2

# Construct a map from input names to Tensor data.
# The graph itself contains inputs for all weight parameters, followed by the input image.
# Since the weights are already embedded, we just need to pass the input image, which is the 
# last input the graph
W = {graph.input[-1]: x.data.numpy()}

# Run the Caffe2 net:
c2_out = prepared.run(W)[0]

# Verify the numerical correctness upto 3 decimal places
np.testing.assert_almost_equal(torch_out.data.cpu().numpy(), c2_out, decimal=3)

### Using the model on mobile devices

So far we have exported a model from PyTorch and shown how to load it and run it in Caffe2. Now that the model is loaded in Caffe2, we can convert it into a format suitable for [running on mobile devices](https://caffe2.ai/docs/mobile-integration.html).

We will use Caffe2's [mobile_exporter](https://github.com/caffe2/caffe2/blob/master/caffe2/python/predictor/mobile_exporter.py) to generate the two model protobufs that can run on mobile. The first is used to initialize the network, and the second actual runs a forward inference step.

In [7]:
# extract the workspace and the graph proto from the internal representation
c2_workspace = prepared.workspace
c2_graph = prepared.predict_net

# Now import the caffe2 mobile exporter
from caffe2.python.predictor import mobile_exporter

# call the Export to get the predict_net, init_net
# init_net, predict_net = mobile_exporter.Export(c2_workspace, c2_graph, c2_graph.input)

<p>Now, on your ios/Android device, you can use the above protobufs and use caffe2::Predictor (iOS) or Caffe2 instance (Android) for deploying them real-time. For more information, also checkout caffe2 https://caffe2.ai/docs/AI-Camera-demo-android.html</p>